
# Assignment 2

**Kacper Kinastowski 30/03/2023**

**Diffusion processes on complex networks**

## TOC:
* [Graph Class docummentation](#class)
* [Class Implementation](#class-imp)
* [Example 1 - how to use this class?](#ex1)
* [Example 2 - name list](#ex2)

## Graph Class docummentation <a class="anchor" id="class"></a>

My goal was to implement undirected graph structure in Python. The structure  

Class **Graph** has the following attributes:

- **vertices** -  A dictionary of vertices and their neighbors. Every vertex is represented by a key in the dictionary and it's value is a list of tuples. Every tuple stores two values, first value represents the neighbor and the second represents the weight of this bond.

For example the graph connection between '*Bob*' and '*Alice*' with bond weight = 0.5 can be represented by:

In [1]:
example_dictionary = {'Bob': [('Alice', 0.5)], 'Alice': [('Bob', 0.5)]}

Class **Graph** has the following methods:

- **__init__(self)** - initialization of graph dictionary.

- **addVertex(self, vert)** - adds vertice to existing graph. Method does not create duplicates.

- **addVerticesFromList(self, vertList)** - adds list of vertices to existing graph.

- **addEdge(self, fromVert, toVert, weight = 1)** - adds edge between two vertices **fromVert** and **toVert** with default **weight = 1**. If given vertices are not present in graph, the method creates them.

- **getVertices()** - Returns a list of all vertices in the graph.

- **getEdges()** - Returns a list of all edges in the graph.

- **getNeighbors(vertKey)** - Returns a list of neighbors of the given vertex **vertKey**. If the vertex does not exist, returns an empty list.

- **__contains__(vert)** - Returns **True** if the given vertex is in the graph, **False** otherwise.

- **saveGraph(filename)** - Saves the graph in the **DOT** language to a file. The file can be used with graph visualization software.

- **shortestDistance(fromVert)** - Returns dictionary with shortest distance between **fromVert** and other vertices in the graph. This method uses Dijkstra's algorithm to find the shortest path from the given vertex to all other vertices in the graph. 

## Class Implementation <a class="anchor" id="class-imp"></a>

In [8]:
import random
import networkx as nx

class Graph:
    
    def __init__(self):
        self.vertices = {}
        
    def addVertex(self, vert):
        if vert not in self.vertices:
            self.vertices[vert] = []
            
    def addVerticesFromList(self, vertList):
        for i in vertList:
            self.addVertex(i)
            
    def addEdge(self, fromVert, toVert, weight = 1):
        
        verts_args = (fromVert, toVert)
        
        for element in verts_args:
            if element not in self.vertices:
                self.addVertex(element)
    
        tup_toVert = (toVert, weight)
        tup_fromVert = (fromVert, weight)
        
        self.vertices[fromVert].append(tup_toVert)
        self.vertices[toVert].append(tup_fromVert)
        
    def getVertices(self):
        return self.vertices.keys()

    def getEdges(self):
        
        edges = []

        for vert in self.vertices:
            for neighbor, weight in self.vertices[vert]:
                if (neighbor, vert, weight) not in edges:
                    edges.append((vert, neighbor, weight))

        return edges
    
    def getNeighbors(self, vertKey):
        if vertKey in self.vertices:
            l = [neigh[0] for neigh in self.vertices[vertKey]]
        else:
            l = []
            
        #delete duplicates
        res = [*set(l)]
        return res
        
    
    def __contains__(self, vert):
        return vert in self.vertices
    
    def saveGraph1(self, filename):
        with open(filename, 'w') as f:
            f.write('graph {\n')
            for vert in self.vertices:
                for neigh in self.vertices[vert]:
                    f.write('\t{} -- {} [label={}];\n'.format(vert, neigh[0], neigh[1]))
            f.write('}\n')
            
    def saveGraph(self, filename, add_label=False):
        with open(filename, 'w') as f:
            f.write('graph {\n')
            visited = set()
            for vert in self.vertices:
                for neigh, weight in self.vertices[vert]:
                    if (vert, neigh) not in visited and (neigh, vert) not in visited:
                        if add_label:
                            f.write('\t{} -- {} [label={}];\n'.format(vert, neigh, weight))
                        else:
                            f.write('\t{} -- {};\n'.format(vert, neigh))
                        visited.add((vert, neigh))
            f.write('}\n')

    def getShortestPaths(self, fromVert):
        distances = {vert: float('inf') for vert in self.vertices}
        distances[fromVert] = 0
        visited = {vert: False for vert in self.vertices}

        while True:
            minDist = float('inf')
            minVert = None
            for vert in self.vertices:
                if not visited[vert] and distances[vert] < minDist:
                    minDist = distances[vert]
                    minVert = vert

            if minVert is None:
                break

            for neigh, weight in self.vertices[minVert]:
                newDist = distances[minVert] + weight
                if newDist < distances[neigh]:
                    distances[neigh] = newDist

            visited[minVert] = True

        return distances
    
    def toNetworkx(self):
        nx_graph = nx.Graph()

        for vertex in self.vertices:
            nx_graph.add_node(vertex)

        for edge in self.getEdges():
            from_vert, to_vert, weight = edge
            nx_graph.add_edge(from_vert, to_vert, weight=weight)

        return nx_graph

    def randomGraph(self, n, p):
        self.vertices = {}
        self.numVertices = 0

        for node in range(n):
            self.addVertex(node)

        edges = [(i, j) for i in range(n) for j in range(i + 1, n) if random.random() < p]

        for i, j in edges:
            self.addEdge(i, j)

        return self

### What's new? <a class="anchor" id="ex1"></a>

Demo of new *randomGraph* method:

In [3]:
graph = Graph()
graph.randomGraph(5, 0.5)
graph.getEdges()

[(0, 1, 1), (0, 4, 1), (1, 2, 1), (1, 4, 1), (2, 3, 1), (2, 4, 1)]

New *toNetworkx method* :

In [7]:
x = graph.toNetworkx()
print(x.edges)

[(0, 1), (0, 4), (1, 2), (1, 4), (2, 3), (2, 4)]
